In [1]:
# Application of FL task
from MLModel import *
from FLModel import *
from utils import *

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [2]:
def load_cnn_mnist(num_users):
    train = datasets.MNIST(root="~/data/", train=True, download=True, transform=transforms.ToTensor())
    train_data = train.data.float().unsqueeze(1)
    train_label = train.targets

    mean = train_data.mean()
    std = train_data.std()
    train_data = (train_data - mean) / std

    test = datasets.MNIST(root="~/data/", train=False, download=True, transform=transforms.ToTensor())
    test_data = test.data.float().unsqueeze(1)
    test_label = test.targets
    test_data = (test_data - mean) / std

    # split MNIST (training set) into non-iid data sets
    non_iid = []
    user_dict = mnist_noniid(train_label, num_users)
    for i in range(num_users):
        idx = user_dict[i]
        d = train_data[idx]
        targets = train_label[idx].float()
        non_iid.append((d, targets))
    non_iid.append((test_data.float(), test_label.float()))
    return non_iid

In [3]:
"""
1. load_data
2. generate clients (step 3)
3. generate aggregator
4. training
"""
client_num = 4
d = load_cnn_mnist(client_num)

In [4]:
"""
FL model parameters.
"""
import warnings
warnings.filterwarnings("ignore")

lr = 0.15

fl_param = {
    'output_size': 10,
    'client_num': client_num,
    'model': MNIST_CNN,
    'data': d,
    'lr': lr,
    'E': 500,
    'C': 1,
    'eps': 4.0,
    'delta': 1e-5,
    'q': 0.01,
    'clip': 0.1,
    'tot_T': 10,
    'batch_size': 128,
    'device': device
}

fl_entity = FLServer(fl_param).to(device)

sigma = 1.0771102905273438


In [5]:
import time

acc = []
start_time = time.time()
for t in range(fl_param['tot_T']):
    acc += [fl_entity.global_update()]
    print("global epochs = {:d}, acc = {:.4f}".format(t+1, acc[-1]), " Time taken: %.2fs" % (time.time() - start_time))

global epochs = 1, acc = 0.5342  Time taken: 302.74s
global epochs = 2, acc = 0.7952  Time taken: 606.97s
global epochs = 3, acc = 0.8969  Time taken: 910.56s
global epochs = 4, acc = 0.9250  Time taken: 1211.34s
global epochs = 5, acc = 0.9406  Time taken: 1516.22s
global epochs = 6, acc = 0.9513  Time taken: 1818.05s
global epochs = 7, acc = 0.9544  Time taken: 2125.24s
global epochs = 8, acc = 0.9579  Time taken: 2427.08s
global epochs = 9, acc = 0.9615  Time taken: 2741.97s
global epochs = 10, acc = 0.9633  Time taken: 3044.31s


In [ ]:
# SGD (mnt=0.9)